# ML Nautanki

In [1]:
from pandas import read_csv, DataFrame
from random import randint

In [2]:
data = DataFrame(read_csv(r'Data.csv'))
data.head()

,ID,Cuisine,Dish,Rating,Value
0,71,Dessert,Rasgulla,4,26
1,50,Dessert,Gulab Jamun,5,25
2,72,Fast Food,Subway,2,28
3,64,Indian,Butter Naan,0,3
4,40,Mughlai,Kebab,3,13


In [3]:
def maxEaten(ID, data):
    return data['Dish'][data['ID'] == ID].value_counts().idxmax()

In [4]:
def collaborativeFiltering(ID, data):
    dishes = data['Dish'][data['ID'] == ID].unique()

    IDs = list()

    for i in data['ID'].unique():
        if set(dishes).issubset(set(data['Dish'][data['ID'] == i].unique())):
            IDs.append(i)

    newData = data[data['ID'].isin(IDs)]

    newDishes = newData['Dish'][-newData['Dish'].isin(dishes)].unique()

    return newData['Dish'][newData['Dish'].isin(newDishes)][newData['ID'].isin(IDs)].value_counts().idxmax()

In [5]:
def valueBased(ID, data):
    sumValue = sum(data['Value'][data['ID'] == ID])

    length = len(data['Value'][data['ID'] == ID])

    roundedValue = round(sumValue / length)

    return data['Dish'][data['Value'] == roundedValue].head(1).item()

In [6]:
def main(ID, data):
    print("Recommendation based on maximum number of dishes eaten: ", maxEaten(ID, data))
    print("\nRecommendation based on collaborative filtering: ", collaborativeFiltering(ID, data))
    print("\nRecommendation based on distance of values: ", valueBased(ID, data))

In [7]:
main(int(input()), data)

Recommendation based on maximum number of dishes eaten:  Idli Sambhar

Recommendation based on collaborative filtering:  Rice Bowl

Recommendation based on distance of values:  Chilli Potato
